In [18]:
import pickle

In [4]:
from transformers import AutoTokenizer, BertModel

In [3]:
from contriever.src.contriever import Contriever, load_retriever

In [1]:
model_path = "contriever/65421994-average-rmin0.05-rmax0.5-T0.05-131072-0.9995-delete-0.1-google_multiberts-seed_6/checkpoint/lastlog"

In [5]:
model, tokenizer, _ = load_retriever(model_path)

In [6]:
tokenizer.model_max_length = 512

In [32]:
inputs = ["who is famous for singing small town girl", "who is a famous girl from a small town", 
          "who is famous for singing i'm your man", "who is famous for singing any way you want it"]

In [33]:
encoded_dict = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

In [34]:
embeddings = model(encoded_dict["input_ids"], attention_mask = encoded_dict["attention_mask"])

In [35]:
score01 = embeddings[0] @ embeddings[1]
score02 = embeddings[0] @ embeddings[2]
score03 = embeddings[0] @ embeddings[3]
score04 = embeddings[1] @ embeddings[3]

In [36]:
score01, score02, score03, score04

(tensor(1.3421, grad_fn=<DotBackward0>),
 tensor(0.9771, grad_fn=<DotBackward0>),
 tensor(0.9208, grad_fn=<DotBackward0>),
 tensor(0.7449, grad_fn=<DotBackward0>))

In [32]:
datapath = "data/biasbios/train.pickle"
with open(datapath, 'rb') as f:
        ds = pickle.load(f)

In [33]:
inputs = []
for r in ds:
    sent = r["text"][r["start"]:]
    inputs.append(sent)

In [177]:
for key, val in encoded_dict.items():
    encoded_dict[key] = val.to_numpy()

AttributeError: 'Tensor' object has no attribute 'to_numpy'

In [143]:
"google/multiberts-seed_".replace("/", "_")

'google_multiberts-seed_'

In [115]:
encoded_dict["input_ids"].shape

torch.Size([255710, 512])

In [119]:
bert = BertModel.from_pretrained("bert-base-uncased")
bert_tok = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [123]:
bert_sample = bert_tok(inputs[:30], padding=True, truncation=True, return_tensors="pt")

In [138]:
bert_input_ids, bert_attns = bert_sample["input_ids"][:10,:], bert_sample["attention_mask"][:10,:]

In [139]:
bert_v = bert(bert_input_ids, attention_mask=bert_attns)

In [154]:
v = []
for i in range(5):
    v.append(bert_v.last_hidden_state[:,0,:])

In [155]:
zzz=torch.cat(v, dim=0)

In [156]:
zzz.shape

torch.Size([50, 768])

In [163]:
test_contriever_tokens = torch.load("data/biasbios/test.tokens_raw_contriever.pt")
test_contriever_tokens_new = torch.load("data/biasbios/test.tokens_raw_contriever_new.pt")

In [168]:
res = (test_contriever_tokens["X"]["input_ids"] == test_contriever_tokens_new["X"]["input_ids"])

In [174]:
torch.all(res)

tensor(True)

In [36]:
contriever = Contriever.from_pretrained("facebook/contriever")
contriever_tokenizer = tokenizer = AutoTokenizer.from_pretrained("facebook/contriever") #Load the associated tokenizer:

Some weights of the model checkpoint at facebook/contriever were not used when initializing Contriever: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing Contriever from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Contriever from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
encoded_dict_2 = contriever_tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

In [38]:
encoded_dict_2["input_ids"].shape

torch.Size([255710, 512])

In [95]:
test = torch.load("data/biasbios/vectors_extracted_from_trained_models/bert-base-uncased/seed_0/dev.vectors_raw_bert_unbatched.pt")

In [160]:
test3 = torch.load("data/biasbios/vectors_extracted_from_trained_models/bert-base-uncased/seed_42/dev.vectors_raw_bert.pt")

In [96]:
x = test["X"]
z = test["z"]

In [101]:
print(x.shape, type(x))
print(z.shape, type(z))

(39369, 768) <class 'numpy.ndarray'>
(39369,) <class 'numpy.ndarray'>


In [102]:
z[z == 'f'] = 1
z[z == 'm'] = 0
z = z.astype(int)
z = torch.tensor(z).long()

In [105]:
x_prime = torch.tensor(x).long()

In [110]:
type(x_prime) == torch.Tensor

True

In [85]:
x2 = test2["X"]
z2 = test2["z"]

In [161]:
x3 = test3["X"]
z3 = test3["z"]


In [162]:
print(x3.shape, type(x3))
print(z3.shape, type(z3))

torch.Size([39369, 768]) <class 'torch.Tensor'>
(39369,) <class 'numpy.ndarray'>


In [98]:
test_c = torch.load("data/biasbios/vectors_extracted_from_trained_models/contriever/seed_1/dev.vectors_raw_contriever_new.pt")

In [99]:
x_c = test_c["X"]
z_c = test_c["z"]

In [109]:
print(x_c.dim())
print(z_c.shape)

2
(39369,)


In [107]:
tds = torch.utils.data.TensorDataset(x_prime,z)

In [20]:
contriever = Contriever.from_pretrained("facebook/contriever") 
contriever_msmarco = Contriever.from_pretrained("facebook/contriever-msmarco")
mcontriever = Contriever.from_pretrained("facebook/mcontriever")
mcontriever_msmarco = Contriever.from_pretrained("facebook/mcontriever-msmarco")

Some weights of the model checkpoint at facebook/contriever were not used when initializing Contriever: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing Contriever from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Contriever from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/contriever-msmarco were not used when initializing Contriever: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing Contriever from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Contriever from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/851 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676M [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("facebook/contriever") #Load the associated tokenizer:

In [4]:
tokenizer_ms = AutoTokenizer.from_pretrained("facebook/contriever-msmarco")

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
tokenizer_multi = AutoTokenizer.from_pretrained("facebook/mcontriever")

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='facebook/contriever', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [6]:
print(tokenizer_ms)

PreTrainedTokenizerFast(name_or_path='facebook/contriever-msmarco', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [8]:
print(tokenizer_multi)

PreTrainedTokenizerFast(name_or_path='facebook/mcontriever', vocab_size=119547, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [10]:
sentences = [
    "Where was Marie Curie born?",
    "Maria Sklodowska, later known as Marie Curie, was born on November 7, 1867.",
    "Born in Paris on 15 May 1859, Pierre Curie was the son of Eugène Curie, a doctor of French Catholic origin from Alsace."
]

In [11]:
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
inputs_ms = tokenizer_ms(sentences, padding=True, truncation=True, return_tensors="pt")

In [21]:
inputs["attention_mask"] == inputs_ms["attention_mask"]

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True]])

In [18]:
inputs_ms

{'input_ids': tensor([[  101,  2073,  2001,  5032, 12731,  7373,  2141,  1029,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  3814, 15315,  4135,  3527,  9333,  2912,  1010,  2101,  2124,
          2004,  5032, 12731,  7373,  1010,  2001,  2141,  2006,  2281,  1021,
          1010,  7517,  1012,   102,     0,     0,     0,     0,     0,     0],
        [  101,  2141,  1999,  3000,  2006,  2321,  2089,  8165,  1010,  5578,
         12731,  7373,  2001,  1996,  2365,  1997,  8207, 12731,  7373,  1010,
          1037,  3460,  1997,  2413,  3234,  4761,  2013, 24922,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0

In [23]:
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
embeddings = contriever(**inputs)

In [14]:
score01 = embeddings[0] @ embeddings[1] #1.0473
score02 = embeddings[0] @ embeddings[2] #1.0095

In [19]:
score01

tensor(1.0473, grad_fn=<DotBackward0>)

In [13]:
import torch

In [31]:
embeddings[0].requires_grad

True

FileNotFoundError: [Errno 2] No such file or directory: 'data/biasbios/tokens_raw_contriever.pt'

In [198]:
p = "contriever/65682284-average-rmin0.05-rmax0.5-T0.05-131072-0.9995-delete-0.1-google_multiberts-seed_20/checkpoint/latest/checkpoint.pth"

In [208]:
p="contriever/65682285-average-rmin0.05-rmax0.5-T0.05-131072-0.9995-delete-0.1-google_multiberts-seed_21/"

In [220]:
p="contriever/65682284-average-rmin0.05-rmax0.5-T0.05-131072-0.9995-delete-0.1-google_multiberts-seed_20/"

In [221]:
p = p + "checkpoint/latest/checkpoint.pth"

In [183]:
from contriever.src import utils

In [ ]:
model, optimizer, scheduler, opt_checkpoint, step = utils.load(
            model_class,
            model_path,
            opt,
            reset_params=False,
        )

In [222]:
checkpoint = torch.load(p, map_location="cpu")